In [27]:
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain.chains import LLMChain
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.exceptions import OutputParserException
from pydantic import BaseModel, Field
import json
import time

In [28]:
with open('system_prompt.txt', 'r') as file:
    system_prompt = file.read()

In [29]:
class StepResponse(BaseModel):
    title: str = Field(description="title of the step")
    content: str = Field(description="content of the step")
    confidence: str = Field(description="model confidence level for current step")
    next_action: str = Field(description="next action performed by the model")

In [30]:
parser = JsonOutputParser(pydantic_object=StepResponse)
# Define the prompt template
template = f"""{system_prompt}
Messages: {{messages}}
"""
# Create the prompt
prompt = ChatPromptTemplate.from_template(template)
print(prompt.input_variables)
model = OllamaLLM(model="llama3.2", temperature=0)
# Build the chain
chain = prompt | model | parser

['messages']


In [31]:
def generate_chain_of_thought(user_message, max_steps=10, confidence_threshold=99):
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_message),
        AIMessage(content="Understood. I will now create a detailed reasoning chain following the given instructions, starting with a thorough problem decomposition.")
    ]

    step_count = 1
    total_thinking_time = 0

    while True:
        start_time = time.time()
        try:
            step_data = chain.invoke({"messages": messages})
        except OutputParserException:
            content = ""
            step_data = {
                "title": "Raw Response",
                "content": content,
                "next_action": "continue"
            }
        end_time = time.time()
        thinking_time = end_time - start_time
        total_thinking_time += thinking_time

        messages.append({"role": "assistant", "content": json.dumps(step_data)})
        print("Next reasoning step: ", step_data["title"])

        if step_data["next_action"].lower().strip() == "final_answer" or step_count > 10:
            break

        step_count += 1

    # Request final answer
    start_time = time.time()
    while True:
        messages.append({
            "role": "user",
            "content": "Please provide the final answer based on your reasoning above.",
        })
        try:
            final_data = chain.invoke({"messages": messages})
            break
        except Exception:
            print("Error encountered while retrieving the final answer. Retrying...")
            continue
    end_time = time.time()
    thinking_time = end_time - start_time
    total_thinking_time += thinking_time

    return final_data["content"], total_thinking_time

In [32]:
user_message = "code hello world in Java"
final_resp, total_time = generate_chain_of_thought(user_message)

Next reasoning step:  Problem Decomposition
Next reasoning step:  Problem Decomposition
Next reasoning step:  Identifying Primary Sources of Energy Consumption
Next reasoning step:  Identifying Primary Sources of Energy Consumption
Next reasoning step:  Identifying Primary Sources of Energy Consumption
Next reasoning step:  Identifying Primary Sources of Energy Consumption
Next reasoning step:  Identifying Primary Sources of Energy Consumption


KeyError: 'title'

In [ ]:
# Print the steps
print(final_resp)
print(f"Total Thinking Time: {total_time:.2f} seconds")